In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [1]:
!pip install pandas

In [2]:
pip install git+https://github.com/palash1992/GEM.git

  Cloning https://github.com/palash1992/GEM.git to /tmp/pip-req-build-4ghvm14m
  Running command git clone -q https://github.com/palash1992/GEM.git /tmp/pip-req-build-4ghvm14m
  Created wheel for gem: filename=gem-1.0.0-cp37-none-any.whl size=32415 sha256=b1dc7e3bd6c03c88fb921941fc5370b4ce6c1e9530ccb59faaa588c40ab3c88f
  Stored in directory: /tmp/pip-ephem-wheel-cache-84bshacq/wheels/dc/52/15/2b124897c6fb5487de2f116477fba30648416532d6fad07716
Successfully built gem


In [3]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/DATA_NETFLOW/preprocessed.csv")
df_reduce = df[['SRC_IP_ADDR','DST_IP_ADDR','PACKETS']]
df_shuffled = df_reduce.sample(frac=1).reset_index(drop=True)
df_reduce = df_shuffled.iloc[:10000]
df_reduce.to_csv("edges.txt",header = False, sep = ' ', index=False)

Index(['DATE', 'DURATION', 'PROTOCOL', 'SRC_IP_ADDR', 'SRC_PORT',
       'DST_IP_ADDR', 'DST_PORT', 'PACKETS', 'BYTES', 'TOS', 'ATTACK_TYPE',
       'TYPE', 'FLAG_A', 'FLAG_P', 'FLAG_R', 'FLAG_S', 'FLAG_F', 'FLAG_0xdb',
       'FLAG_0xc2', 'FLAG_0xd7', 'FLAG_0xd6', 'FLAG_0xdf', 'FLAG_0xda',
       'FLAG_0x52', 'FLAG_0xd3', 'FLAG_0x5b', 'FLAG_0x5a', 'FLAG_0xd2',
       'FLAG_0xde', 'FLAG_0x53', 'FLAG_0xc6'],
      dtype='object')


In [1]:
import os
path = os.getcwd()
path = path +'/gem'
try:
    os.mkdir(path)
    os.mkdir(path+'/c_exe')
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
#-------------------------------------------------------------------------------
path = path +'/intermediate'
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /content/gem 
Successfully created the directory /content/gem/intermediate 


In [ ]:
import matplotlib.pyplot as plt

from gem.utils import graph_util, plot_util
from gem.evaluation import visualize_embedding as viz
from gem.evaluation import evaluate_graph_reconstruction as gr
from time import time

from gem.embedding.gf import GraphFactorization
from gem.embedding.hope import HOPE
from gem.embedding.lap import LaplacianEigenmaps
from gem.embedding.lle import LocallyLinearEmbedding
from gem.embedding.node2vec import node2vec
from gem.embedding.sdne import SDNE

edge_f = '/content/edges.txt'
isDirected = True

# Load graph
G = graph_util.loadGraphFromEdgeListTxt(edge_f, directed=isDirected)
G = G.to_directed()

models = []
models.append(GraphFactorization(d=2, max_iter=100000, eta=1*10**-4, regu=1.0,
                                 data_set='karate'))

for embedding in models:
    print ('Num nodes: %d, num edges: %d' % (G.number_of_nodes(), G.number_of_edges()))
    t1 = time()

    # Learn embedding - accepts a networkx graph or file with edge list
    Y, t = embedding.learn_embedding(graph=G, edge_f=None, is_weighted=False, no_python=True)
    print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))

    # Evaluate on graph reconstruction
    MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G, embedding, Y, None)
    #---------------------------------------------------------------------------------
    print(("\tMAP: {} \t precision curve: {}\n\n\n\n"+'-'*100).format(MAP,prec_curv[:5]))
    #---------------------------------------------------------------------------------
    # Visualize
    viz.plot_embedding2D(embedding.get_embedding(), di_graph=G, node_colors=None)
    plt.show()